--- Tweeter parser ---

In [341]:
import tweepy
from tweepy import OAuthHandler
 
consumer_key = 'r8BGSocOtv6vZBnjcwkU5qM4u'
consumer_secret = 'CNAJg23IWXLvUUyJfuFUQOYfzvtKfQVQu6SsY82yJY7GGdhBDC'
access_token = '796083347315367936-grZPgV2cW8DvrozevvihPIJ1Oq9WQVK'
access_secret = 'oJoGFZYDDOXjyG30v0FBChWmGfF9RG6fXvDCVuNrDZJGb'
 
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
 
api = tweepy.API(auth)

In [342]:
#def process_or_store(tweet):
#    print(json.dumps(tweet))

In [343]:
#import json
#for tweet in tweepy.Cursor(api.user_timeline).items():
#    process_or_store(tweet._json)

In [ ]:
from tweepy import Stream
from tweepy.streaming import StreamListener
 
class MyListener(StreamListener):
    def on_data(self, data):
        try:
            with open('negative.json', 'a') as f:
                f.write(data)
                return True
        except BaseException as e:
            print('&quot; Error on_data: %s&quot; % str(e)')
            return True
    def on_error(self, status):
        print(status)
        return True
    
twitter_stream = Stream(auth, MyListener())

#--- filter for positive tweets 
#twitter_stream.filter(track=[':)', ':-)', '=)', ':D', ';-)', ';)', '(-;', '(-:', '(:', '(;', '(=', ':d', ':p'], languages=['fr'])

#--- filter for negative tweets
twitter_stream.filter(track=[':(', ':-(', '=(', ')-:', '):', ');', ')='], languages=['fr'])

--- Preprocessing part ---

In [224]:
from nltk.corpus import stopwords
import string

smiles_negativ = [':(', ':-(', '=(', ')-:', '):', ');', ')=']
smiles_positiv = [':)', ':-)', '=)', ':D', ';-)', ';)', '(-;', '(-:', '(:', '(;', '(=', ':d', ':p']
pronoms = ['je', 'tu', 'nous', 'vous', 'il', 'elle', 'ils', 'elles', 'ça', 'jsuis']
smiles = smiles_negativ + smiles_positiv
punctuation = list(string.punctuation)
stop = stopwords.words('french') + punctuation + smiles + ['rt', 'les', '…', 'é', 'ca', 'ée', 'cette', '’', 'a', 'leurs', ''] + pronoms

In [320]:
import re
 
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:pas+\s+[\w_]+[\w_]+[\w_]+)", # concat pas with the following word
    r"(?:[\w_]+\s+pas)", # concat pas with the preceding word
    r"(?:très\s+[\w_]+)", # concat très + word
    r"(?:trop\s+[\w_]+)", # trop + word
    r"(?:n'[\w_]+)", # n' + word
    #r"(?:ne\s+[\w_][^'][\w_]*)",
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z\-_]+[a-z])", # words with - 
    #r"(?:[^jtm]+'[a-z]+)", # words with '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 

In [321]:
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import FrenchStemmer

def stem_tokens(tokens):
    stemmer = FrenchStemmer()
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def preprocess(s):
    tokens = tokens_re.findall(s)
    tokens = [term.lower() for term in tokens if term.lower() 
                not in stop 
                and not term.isdigit() 
                and not term.startswith(('@', '#', 'http'))]
    #tokens = stem_tokens(tokens)
    return tokens

In [ ]:
#tweet = "RT C'est pas ici @hoho: je ne t'aime pas ! trop intelligent je ne veux pas y aller :D http://hoho.com #NLP"
#preprocess(tweet)

--- Tweets' cleaning and tokenizing ---

In [323]:
import json
import operator
from nltk import bigrams
from collections import Counter

tweets_text = []
mood = []
count_all = Counter()

for line in open('positive.json'):
  try: 
    new_tweet = json.loads(line)
    text = new_tweet['text']
    tweets_text.append(text)
    mood.append(1)
  except:
    pass

In [324]:
import json
import operator
from nltk import bigrams
from collections import Counter

for line in open('negative.json'):
  try: 
    new_tweet = json.loads(line)
    text = new_tweet['text']
    tweets_text.append(text)
    mood.append(0)
  except:
    pass

--- Tf-Idf matrix ---

In [325]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(tokenizer=preprocess)
A = tfidf.fit_transform(tweets_text)
A = A.toarray()

In [326]:
import numpy as np
np.unique(A)

array([ 0.        ,  0.09268701,  0.11435201, ...,  0.95361125,
        0.97844342,  1.        ])

In [327]:
U, S, V = np.linalg.svd(A)
print (U.shape)
print (S.shape)
print (V.shape)

(4917, 4917)
(4917,)
(7324, 7324)


In [332]:
V2 = V[:500,:]
M = np.dot(A,V2.transpose())
M.shape

(4917, 500)

--- Algorithmic testing part ---

In [333]:
from sklearn.ensemble import RandomForestRegressor as rf
from sklearn.cross_validation import train_test_split

X = M
Y = mood

X_train, X_test, y_train, y_test = train_test_split(X,Y)

In [334]:
model = rf()
model.fit(X_train,y_train)
pred = model.predict(X_test)
pred = np.round(pred)

In [335]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
r2_score(y_test, pred)

0.19099717659137594

In [ ]:
#for var_text in tweets_text:
#    tweet_prep = preprocess(var_text)
#    print(tweet_prep)
    #f.write(joined_terms)
    #terms_bigram = bigrams(terms_all)
    #print(terms_all)
    #count_all.update(terms_all)
#f.close()
#print(count_all.most_common(100))